In [8]:
import pandas as pd
import pickle

In [9]:
rating = pd.read_csv('../data/raw/u.data', sep="\t", header=None)
rating.columns = ['user_id', 'item_id', 'rating', 'timestamp']


In [10]:
# Sort per user by time
rating = rating.sort_values(['user_id', 'timestamp'])

In [11]:
train_list = []
val_list = []
test_list = []

for user_id, user_df in rating.groupby('user_id'):
    n = len(user_df)
    if n < 5:
        # very small users → all to train
        train_list.append(user_df)
        continue

    train_end = int(0.7 * n)
    val_end = int(0.85 * n)

    train_list.append(user_df.iloc[:train_end])
    val_list.append(user_df.iloc[train_end:val_end])
    test_list.append(user_df.iloc[val_end:])

In [12]:
train = pd.concat(train_list)
val = pd.concat(val_list)
test = pd.concat(test_list)

In [13]:
# Build user-item map ONLY from train
user_item_map = train.pivot(
    index='user_id',
    columns='item_id',
    values='rating'
)

In [14]:
# Save
with open("../data/processed/user_item_map.pkl", "wb") as f:
    pickle.dump(user_item_map, f)

with open("../data/processed/train.pkl", "wb") as f:
    pickle.dump(train, f)

with open("../data/processed/val.pkl", "wb") as f:
    pickle.dump(val, f)

with open("../data/processed/test.pkl", "wb") as f:
    pickle.dump(test, f)